In [1]:
import sys
from PIL import Image
def concat_images(image_paths,output_path,margin):
  # images = [Image.open(x) for x in ['profile.jpg', 'profile.jpg', 'profile.jpg']]
  images = [Image.open(x) for x in image_paths]
  widths, heights = zip(*(i.size for i in images))

  total_width = sum(widths)+margin
  max_height = max(heights)+margin

  new_im = Image.new('RGB', (total_width, max_height),(255,255,255,0))

  x_offset = int(margin/2)
  for im in images:
    new_im.paste(im, (x_offset,int(margin/2)))
    x_offset += im.size[0]

  new_im.save(output_path)


In [2]:
import cv2
import numpy as np
import random
import numpy as np

def create_image(card_number,output_without_back,margin,card_type):
  output_path = []
  font = ""
  font_rnd = random.randint(0,1)
  if(font_rnd!=0):
    font=str(font_rnd)
  for num in card_number:
    if(num==" "):
      output_path.append("empty.png")
    elif(num=="-"):
      output_path.append("dash.png")
    else:
      if(card_type=="national"):
        output_path.append(num+card_type+".png")
      else:
        output_path.append(num+card_type+font+".png")

  concat_images(output_path,output_without_back,margin)

def rotateImage(image, angle):
  (h, w) = image.shape[:2]
  (cX, cY) = (w // 2, h // 2)
  M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
  rotated = cv2.warpAffine(image, M, (w, h),                           
                           borderValue=(255, 255, 255))
  return rotated    

def generate_data(model_input,margin,card_type):
  create_image(model_input,"without.jpg",margin,card_type)
  img = cv2.imread("without.jpg")
  img = rotateImage(img,random.randint(-3,3))
  rnd_back = random.randint(0,2)
  back = cv2.imread(card_type+"back"+str(rnd_back)+".png")
  char_width = (len(img[0])-margin)/len(model_input)

  random_nums = []
  for j in range(len(img[0])):
    if(j>int(margin/2)):
      if((j-int(margin/2))%char_width==0):
        random_nums.append(random.randint(0, 100))
  rnd = 0
  random_padding = random.randint(150,300)
  for i in range(len(img)):
    for j in range(len(img[i])):
      if(j>int(margin/2)):
        if((j-int(margin/2))%char_width==0):
          index = int((j-int(margin/2))/char_width)-1        
          rnd=random_nums[index]
      avg = sum(img[i][j]) / 3
      if(avg>230):
        img[i][j]=back[(i+random_padding)%len(back)][(j+random_padding)%len(back[0])]
      else:            
        img[i][j][0]=(img[i][j][0]+rnd)%256
        img[i][j][1]=(img[i][j][1]+rnd)%256
        img[i][j][2]=(img[i][j][2]+rnd)%256 
  model_input = model_input.replace(" ","")
  model_input = model_input.replace("-","") 

  final_width = 200
  scale_percent = final_width / len(img[0])
  width = int(img.shape[1] * scale_percent )
  height = int(img.shape[0] * scale_percent )
  dim = (width, height)
  
  # resize image
  img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
  
  cv2.imwrite("/content/1" + card_type+"/"+model_input+".jpg", img)
        
# model_input = "3489 2109 3910 3867"
# generate_data(model_input,50,"bank")
# generate_data("006-706257-1",50,"national")



In [3]:
!unzip national.zip -d 1national

data_size = 1000
for i in range(data_size):
  s = ""
  if(i%2==1):
    for j in range(12):
      if(j==3):
        s+="-"
      elif(j==10):
        s+="-"    
      else:
        rnd = random.randint(0,9)
        s+=str(rnd)
  else:
    for j in range(10):
      rnd = random.randint(0,9)
      s+=str(rnd)
  margin=random.randint(50,150)
  generate_data(s,margin,"national")
!zip -r /content/national_test_dataset.zip /content/1national

Archive:  national.zip
  inflating: 1national/nationalback.png  
  adding: content/1national/ (stored 0%)
  adding: content/1national/9158597152.jpg (deflated 3%)
  adding: content/1national/0955652812.jpg (deflated 3%)
  adding: content/1national/5947177966.jpg (deflated 3%)
  adding: content/1national/9321122036.jpg (deflated 3%)
  adding: content/1national/5174864264.jpg (deflated 3%)
  adding: content/1national/0352229272.jpg (deflated 2%)
  adding: content/1national/0890747301.jpg (deflated 3%)
  adding: content/1national/1436303217.jpg (deflated 4%)
  adding: content/1national/7557254512.jpg (deflated 3%)
  adding: content/1national/8457699260.jpg (deflated 2%)
  adding: content/1national/7686482087.jpg (deflated 3%)
  adding: content/1national/2859331314.jpg (deflated 3%)
  adding: content/1national/1446532398.jpg (deflated 3%)
  adding: content/1national/2696549529.jpg (deflated 3%)
  adding: content/1national/7073921236.jpg (deflated 3%)
  adding: content/1national/6966693510.

In [4]:
!unzip bank.zip -d 1bank

data_size = 1000
for i in range(data_size):
  s = ""
  for j in range(19):
    if(j==4):
      s+=" "
    elif(j==9):
      s+=" "    
    elif(j==14):
      s+=" "    
    else:
      rnd = random.randint(0,9)
      s+=str(rnd)
  margin=random.randint(50,150)
  generate_data(s,margin,"bank")

!zip -r /content/bank_test_dataset.zip /content/1bank


Archive:  bank.zip
  inflating: 1bank/bankback.png      
  adding: content/1bank/ (stored 0%)
  adding: content/1bank/4061194550543152.jpg (deflated 4%)
  adding: content/1bank/4804239988258801.jpg (deflated 4%)
  adding: content/1bank/6396140459315343.jpg (deflated 3%)
  adding: content/1bank/1188730586345174.jpg (deflated 4%)
  adding: content/1bank/8973837262220735.jpg (deflated 4%)
  adding: content/1bank/5797415987923194.jpg (deflated 4%)
  adding: content/1bank/1207464280833775.jpg (deflated 4%)
  adding: content/1bank/1369013156149311.jpg (deflated 4%)
  adding: content/1bank/6894183254754462.jpg (deflated 4%)
  adding: content/1bank/5068262410786265.jpg (deflated 4%)
  adding: content/1bank/6816120995822160.jpg (deflated 4%)
  adding: content/1bank/6931993539003031.jpg (deflated 4%)
  adding: content/1bank/2611172128624267.jpg (deflated 4%)
  adding: content/1bank/6995883967007443.jpg (deflated 4%)
  adding: content/1bank/9467159650006542.jpg (deflated 4%)
  adding: content/1ba